In [26]:
import torch

# Загрузка модели YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'custom', path='models/bag_belt_yolo.pt')
# Установка порога по уверенности и подавления
model.conf = 0.15  # Порог по уверенности
model.iou = 0.4  # Порог подавления



Using cache found in C:\Users\Дмитрий/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-21 Python-3.8.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080 Ti Laptop GPU, 16384MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20873139 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [45]:
def download_models(folder_name='models'):
    '''
    Функция для загрузки моделей обученных нейронных сетей в папку folder_name 
    из моего гугл диска
    '''
    #if not os.path.exists(folder_name + ''):
    # Создание пустой папки 
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

    print('Загружаю модель нейронной сети:')
    url_1 = "https://drive.google.com/uc?id=1p2ErNFGtcN9Gly3ewy1UAz68wAKxSFW3"
    output_1 = folder_name + "/bag_belt_yolo.pt"

    gdown.download(url_1, output_1, quiet=False)


In [40]:
import gdown

In [46]:
download_models()

Загружаю модель нейронной сети:


Downloading...
From (uriginal): https://drive.google.com/uc?id=1p2ErNFGtcN9Gly3ewy1UAz68wAKxSFW3
From (redirected): https://drive.google.com/uc?id=1p2ErNFGtcN9Gly3ewy1UAz68wAKxSFW3&confirm=t&uuid=e19753ee-29d5-4496-9685-402dea92c4b8
To: c:\WORK_ЦТиП\Koldim2001 repo\annotation_making_cvat\models\model_human_detection_final.pth
100%|██████████| 42.2M/42.2M [00:02<00:00, 17.5MB/s]


In [27]:
def save_annotations_to_yolo_format(output_file, results, height_max, width_max):
    with open(output_file, 'w') as f:
        for result in results.xyxy[0]:
            label = result[-1]
            xmin, ymin, xmax, ymax = result[:4]

            # Получение координат центра объекта и его ширины и высоты
            width = xmax - xmin
            height = ymax - ymin
            x_center = xmin + width / 2
            y_center = ymin + height / 2

            # Запись аннотации в формате YOLO 1.1
            label = int(label)
            x_center_formatted = '{:.6f}'.format(x_center.item() / width_max)
            y_center_formatted = '{:.6f}'.format(y_center.item() / height_max)
            width_formatted = '{:.6f}'.format(width.item() / width_max)
            height_formatted = '{:.6f}'.format(height.item() / height_max)

            line = f"{label} {x_center_formatted} {y_center_formatted} {width_formatted} {height_formatted}\n"
            f.write(line)

In [28]:
import os
import cv2

image_folder = 'image_folder'
output_folder = 'results/obj_train_data'

# Создание папки для сохранения аннотаций, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Обработка изображений и сохранение аннотаций в формате YOLO 1.1
for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height_max, width_max = image.shape[0], image.shape[1]

    # Инференс с использованием модели YOLOv5
    results = model(image)

    # Путь к файлу аннотации
    annotation_file = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}.txt")

    # Сохранение аннотаций в формате YOLO 1.1
    save_annotations_to_yolo_format(annotation_file, results, height_max, width_max)

print(f"Аннотации сохранены")

train_file = 'results/train.txt'  # Имя файла train.txt

# Создание пустого файла train.txt, если он не существует
if not os.path.exists(train_file):
    open(train_file, 'w').close()

# Запись имен файлов в train.txt
with open(train_file, 'w') as f:
    for image_file in os.listdir(image_folder):
        f.write('data/obj_train_data/' + image_file + '\n')


Аннотации сохранены


In [29]:
import shutil

def copy_files_to_folder(source_folder, destination_folder):
    # Создание пустой папки
    os.makedirs(destination_folder, exist_ok=True)

    # Копирование файлов в папку
    for file_name in os.listdir(source_folder):
        source_file = os.path.join(source_folder, file_name)
        destination_file = os.path.join(destination_folder, file_name)
        shutil.copy2(source_file, destination_file)

def delete_folder(folder):
    # Удаление папки со всем содержимым
    shutil.rmtree(folder)

# Указать путь к папке, из которой нужно скопировать файлы
source_folder = 'extra_data'
# Указать путь к папке, в которую нужно скопировать файлы
destination_folder = 'results'

# Копирование файлов в папку
copy_files_to_folder(source_folder, destination_folder)



In [30]:
import zipfile

def create_zip_archive(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, arcname=os.path.relpath(file_path, folder_path))

# Указать путь к папке, файлы которой нужно добавить в zip-архив
folder_path = 'results'
# Указать путь для сохранения zip-архива
output_path = 'annotations.zip'

create_zip_archive(folder_path, output_path)

In [31]:
# Удаление папки со всем содержимым
delete_folder(destination_folder)

In [ ]:
https://drive.google.com/file/d/1p2ErNFGtcN9Gly3ewy1UAz68wAKxSFW3/view?usp=sharing